In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('../data/Tweets.csv', usecols=['text', 'selected_text', 'sentiment'])
pd.set_option('display.max_columns', None)

for col in df.columns:
    df[col] = df[col].str.lower().str.strip().str.split(' ')

print(df.head())

                                                text  \
0        [i`d, have, responded,, if, i, were, going]   
1  [sooo, sad, i, will, miss, you, here, in, san,...   
2                    [my, boss, is, bullying, me...]   
3               [what, interview!, leave, me, alone]   
4  [sons, of, ****,, why, couldn`t, they, put, th...   

                                 selected_text   sentiment  
0  [i`d, have, responded,, if, i, were, going]   [neutral]  
1                                  [sooo, sad]  [negative]  
2                               [bullying, me]  [negative]  
3                           [leave, me, alone]  [negative]  
4                            [sons, of, ****,]  [negative]  


In [98]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [38]:
df = df.drop(df[df['selected_text'].isna()].index.tolist())
print(df.isna().sum())

text             0
selected_text    0
sentiment        0
dtype: int64


In [74]:
df_fulltrain, df_test = train_test_split(df[['selected_text', 'sentiment']], test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_fulltrain, test_size=0.25, random_state=1)

len(df_train), len(df_val), len(df_test), len(df)

(16488, 5496, 5496, 27480)

In [75]:
df_train.shape, df_val.shape, df_test.shape

((16488, 2), (5496, 2), (5496, 2))

In [76]:
df_train.columns

Index(['selected_text', 'sentiment'], dtype='object')

In [77]:
y_train = df_train.sentiment.values
y_val = df_val.sentiment.values
y_test = df_test.sentiment.values

del df_train['sentiment']
del df_val['sentiment']
del df_test['sentiment']

In [78]:
df.sentiment.value_counts()

[neutral]     11117
[positive]     8582
[negative]     7781
Name: sentiment, dtype: int64